# helper_functions.ipynb

These are backend functions to help login, query, and store Plaid API data

In [2]:
import requests, json, uuid, nbformat, datetime, os, psycopg2, pdb, platform, sys, socket, psutil, datetime
import pandas as pd

PLAID_COUNTRY_CODES = ['CA','US']
PLAID_PRODUCTS = ['transactions']
PLAID_CLIENT_ID= os.environ['PLAID_CLIENT_ID']
PLAID_SECRET= os.environ['PLAID_SECRET']
PLAID_ENV = os.environ['PLAID_ENV']
PLAID_BASE_URL = f'https://{PLAID_ENV}.plaid.com'

POSTGRES_HOST= "db"
POSTGRES_USER= os.environ['POSTGRES_USER']
POSTGRES_PASSWORD= os.environ['POSTGRES_PASSWORD']


## API Calls 

In [2]:
def plaid_post(endpoint, payload) -> json:
    """ 
    Helper function to make POST requests to Plaid API
    """
    url = f"{PLAID_BASE_URL}/{endpoint}"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    return response.json()

def get_account(access_token):
    payload = {
        "client_id": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
        "access_token": access_token
    }
    accounts_response = plaid_post("accounts/get", payload)
    return accounts_response

def get_account_transactions(access_token):
    try:
        transactions = []
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "access_token": access_token,
            "start_date": "2024-01-01",
            "end_date": "2024-12-31",
            "options": {
                "count": 100,  
                "offset": 0   
            }
        }

        while True:
            transactions_response = plaid_post("transactions/get", payload)
            transactions.extend(transactions_response.get("transactions", []))
            
            if len(transactions) >= transactions_response.get("total_transactions", 0):
                break
            payload["options"]["offset"] += 100

        return transactions
    except Exception as e:
        print (f"There was an error. Please report outputs of this cell to the developer:\n {e}, \n Response: {transactions_response}")



## Dataframe Transformation

In [ ]:
def get_account_df(accounts_response) -> pd.DataFrame:
   """
   Returns the accounting information in a pandas data frame
   """
   return pd.json_normalize(accounts_response, sep='_')


def get_accounts_df():
    accounts_df_list = []
    
    for single_access_token in access_tokens:
        a = get_account_df(get_account(single_access_token[0]))
        print(a.head())  # Show the first few rows
        display(a)
        accounts_df_list.append(a)
    pdb.set_trace()
    merged_accounts_df = pd.concat(accounts_df_list, ignore_index=True)
    pdb.set_trace()

    return merged_accounts_df


def get_transactions_df():
    transactions_list = []
    for single_access_token in access_tokens:
        transactions_list.extend(get_account_transactions(single_access_token[0]))
    
    return pd.json_normalize(transactions_list)


## Database 


In [ ]:
def db_conn()-> psycopg2.extensions.connection:
    """
    Instantiate connection to database
    """
    return psycopg2.connect(
        host=POSTGRES_HOST,
        database="finances",
        user="admin",
        password="admin"
    )

def get_stored_public_access_tokens():
    """
    Retrieves public access tokens from local DB
    """
    try: 
        db = db_conn()
        cur = db.cursor()
        cur.execute("SELECT DISTINCT plaid_access_token FROM accounts;")
        tokens = cur.fetchall()
        cur.close()

        return tokens

    except Exception as e:
        print(f"Error: {e}")
    finally:
        db.close()
        print(f"Found {len(tokens)} accounts and their access tokens.")

    
def insert_account_df(access_token, accounts_response):
    """
    Inserts account information and access_token into database
    """
    accounts_df = get_account_df(accounts_response['accounts'])

    db = db_conn()
    cur = db.cursor()

    for _, account in accounts_df.iterrows():
        cur.execute("""
            INSERT INTO accounts (
                account_id, 
                mask,
                name,
                official_name,
                persistent_account_id,
                subtype,
                type,
                user_email,
                user_phone,
                plaid_access_token
            ) 
            VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            ON CONFLICT (account_id) 
            DO UPDATE SET
                mask = EXCLUDED.mask,
                name = EXCLUDED.name,
                official_name = EXCLUDED.official_name,
                persistent_account_id = EXCLUDED.persistent_account_id,
                subtype = EXCLUDED.subtype,
                type = EXCLUDED.type,
                user_email = EXCLUDED.user_email,
                user_phone = EXCLUDED.user_phone,
                plaid_access_token = EXCLUDED.plaid_access_token;
        """, (
            account['account_id'],
            account['mask'],
            account['name'],
            account['official_name'],
            account['persistent_account_id'],
            account['subtype'],
            account['type'],
            email,
            phone,
            access_token,
        ))

    db.commit()
    cur.close()
    db.close()
    print(f"Successfully updated {accounts_df.count()} accounts into database.")

import datetime

def insert_transactions_df(transactions_df):
    db = db_conn()
    cur = db.cursor()
    
    for _, transaction in transactions_df.iterrows():
        cur.execute("""
            INSERT INTO transactions (
                transaction_id,
                account_id,
                amount,
                authorized_date,
                category_id,
                date,
                iso_currency_code,
                logo_url,
                merchant_entity_id,
                merchant_name,
                name,
                payment_channel,
                pending,
                personal_finance_category_icon_url,
                website,
                personal_finance_category_detailed,
                personal_finance_category
            ) 
            VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            ON CONFLICT (transaction_id) 
            DO UPDATE SET
                account_id = EXCLUDED.account_id,
                amount = EXCLUDED.amount,
                authorized_date = EXCLUDED.authorized_date,
                category_id = EXCLUDED.category_id,
                date = EXCLUDED.date,
                iso_currency_code = EXCLUDED.iso_currency_code,
                logo_url = EXCLUDED.logo_url,
                merchant_entity_id = EXCLUDED.merchant_entity_id,
                merchant_name = EXCLUDED.merchant_name,
                name = EXCLUDED.name,
                payment_channel = EXCLUDED.payment_channel,
                pending = EXCLUDED.pending,
                personal_finance_category_icon_url = EXCLUDED.personal_finance_category_icon_url,
                website = EXCLUDED.website,
                personal_finance_category_detailed = EXCLUDED.personal_finance_category_detailed,
                personal_finance_category = EXCLUDED.personal_finance_category;
        """, (
            transaction['transaction_id'],
            transaction['account_id'],
            transaction['amount'],
            transaction['authorized_date'] if pd.notnull(transaction['authorized_date']) else None,
            transaction['category_id'],
            transaction['date'],
            transaction['iso_currency_code'],
            transaction['logo_url'] if pd.notnull(transaction['logo_url']) else None,
            transaction['merchant_entity_id'] if pd.notnull(transaction['merchant_entity_id']) else None,
            transaction['merchant_name'] if pd.notnull(transaction['merchant_name']) else None,
            transaction['name'],
            transaction['payment_channel'],
            transaction['pending'],
            transaction['personal_finance_category_icon_url'] if pd.notnull(transaction['personal_finance_category_icon_url']) else None,
            transaction['website'] if pd.notnull(transaction['website']) else None,
            transaction['personal_finance_category.detailed'],
            transaction['personal_finance_category.primary']
        ))
    db.commit()
    cur.close()
    print(f"Successfully inserted transactions {transactions_df.count()} into database.")

    
def upsert_account_balances_df(accounts_df):

    db = db_conn()
    cur = db.cursor()
    for _, account in accounts_df.iterrows():
        cur.execute("""
            INSERT INTO accounts_balance_history (
                account_id, 
                balances_available,
                balances_current,
                balances_iso_currency_code,
                balances_limit,
                balances_unofficial_currency_code,
                balances_datetime
            ) 
            VALUES (
                %s, %s, %s, %s, %s, %s, %s
            );
        """, (
            account['account_id'],
            account['balances_available'],
            account['balances_current'],
            account['balances_iso_currency_code'],
            account['balances_limit'],
            account['balances_unofficial_currency_code'],
            datetime.datetime.now(),
        ))

    db.commit()
    cur.close()
    db.close()

    print(f"Successfully updated balances and history {accounts_df.count()} into database.")


## Authentication

In [4]:
def generate_link_token():
    """
    Generate link token to authenticate with Plaid API
    """
    try:
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "client_name": "Juypter Notebook",
            "country_codes": PLAID_COUNTRY_CODES,
            "language": "en",
            "user": {"client_user_id": str(uuid.uuid4()), "phone_number":phone, "email_address": email},
            "hosted_link": {},
            "products": PLAID_PRODUCTS
        }
        link_token_response = plaid_post("link/token/create", payload)
        print("Navigate to this page and authenticate with your bank: ", link_token_response["hosted_link_url"], \
            "\nThis link expires", link_token_response['expiration'])
    except Exception as e:
        print (f"There was an error. Please report outputs of this cell to the developer:\n {e}, \n Response: {link_token_response}")
    return link_token_response['link_token']

def generate_and_save_public_token(link_token):
    try:
        payload = {
        "client_id": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
        "link_token": link_token
        }
        link_token_details = plaid_post("link/token/get", payload)

        # Exchange Public Token for Access Token (Assumes Plaid Link flow completed)

        public_token = link_token_details['link_sessions'][0]['results']['item_add_results'][0]['public_token'] 
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "public_token": public_token
        }

        exchange_response = plaid_post("item/public_token/exchange", payload)
        access_token = exchange_response.get("access_token")
        print("Access token has been generated successfully")   
    except:
        print (f"There was an error. Please report outputs of this cell to the developer:\n {e}, \n Response: {exchange_response}")

    accounts_response = get_account(access_token)
    insert_account_df(access_token, accounts_response)


## Tasks

In [ ]:
def get_and_save_all_account_transactions():
    transactions_df = get_transactions_df()
    insert_transactions_df(transactions_df)


def get_and_save_balance_history():
    accounts_df = get_accounts_df()
    pdb.set_trace()

    upsert_account_balances_df(accounts_df)

In [5]:

def about():
    print("="*60)
    print("="*60)
    print(f"Jupyter Finances")
    # Date/Time
    print(f"Current Date and Time: {datetime.datetime.now()}")
    
    print("="*60)
    print(f"Plaid API ({PLAID_ENV})")
    print("="*60)
    print(f"PLAID_CLIENT_ID: {PLAID_CLIENT_ID}")
    print(f"PLAID_PRODUCTS: {PLAID_PRODUCTS}")
    print(f"PLAID_COUNTRY_CODES: {PLAID_COUNTRY_CODES}")
    print("="*60)
